In [48]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
import re
from bs4 import BeautifulSoup
import contractions
import warnings
 

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/xiaoyuhang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xiaoyuhang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/xiaoyuhang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [49]:
! pip install bs4 # in case you don't have it installed
! pip install contractions # in case you don't have it installed
! pip install scikit-learn # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz

## Read Data

In [50]:
dtype={7: object}
url = 'https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz'
data = pd.read_csv(url, sep='\t', on_bad_lines='skip', dtype=dtype)
# path = 'amazon_reviews_us_Office_Products_v1_00.tsv.gz'
# data = pd.read_csv(path, sep='\t', on_bad_lines='skip', dtype=dtype)
# unzipped_path = 'amazon_reviews_us_Office_Products_v1_00.tsv'
# data = pd.read_csv(unzipped_path, sep='\t', on_bad_lines='skip', dtype=dtype)
print("Rows: ", data.shape[0])
data.head()

# def cannot_convert_to_float(x):
#     try:
#         float(x)  # Try converting to float
#         return False
#     except ValueError:  # If conversion fails, return False
#         return True

# column_data = data.iloc[:, 7]
# unique_types = set(map(type, column_data))
# print(unique_types)
# filtered_data = column_data.apply(lambda x: x if isinstance(x, (float, str)) else None)
# filtered_data = column_data.apply(lambda x: x if isinstance(x, float) and x % 1 != 0 else None)
# filtered_data = column_data.apply(lambda x: x if isinstance(x, str) and (cannot_convert_to_float(x) or float(x) % 1 != 0) else None)
# print(filtered_data.dropna())

Rows:  2640254


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,43081963,R18RVCKGH1SSI9,B001BM2MAC,307809868,"Scotch Cushion Wrap 7961, 12 Inches x 100 Feet",Office Products,5,0.0,0.0,N,Y,Five Stars,Great product.,2015-08-31
1,US,10951564,R3L4L6LW1PUOFY,B00DZYEXPQ,75004341,"Dust-Off Compressed Gas Duster, Pack of 4",Office Products,5,0.0,1.0,N,Y,"Phffffffft, Phfffffft. Lots of air, and it's C...",What's to say about this commodity item except...,2015-08-31
2,US,21143145,R2J8AWXWTDX2TF,B00RTMUHDW,529689027,Amram Tagger Standard Tag Attaching Tagging Gu...,Office Products,5,0.0,0.0,N,Y,but I am sure I will like it.,"Haven't used yet, but I am sure I will like it.",2015-08-31
3,US,52782374,R1PR37BR7G3M6A,B00D7H8XB6,868449945,AmazonBasics 12-Sheet High-Security Micro-Cut ...,Office Products,1,2.0,3.0,N,Y,and the shredder was dirty and the bin was par...,Although this was labeled as &#34;new&#34; the...,2015-08-31
4,US,24045652,R3BDDDZMZBZDPU,B001XCWP34,33521401,"Derwent Colored Pencils, Inktense Ink Pencils,...",Office Products,4,0.0,0.0,N,Y,Four Stars,Gorgeous colors and easy to use,2015-08-31


## Keep Reviews and Ratings

In [51]:
data = data[['star_rating', 'review_headline', 'review_body']]
data.dropna(inplace=True)
print("Rows: ", data.shape[0])
print("Three sample reviews:")
data.sample(3, random_state=1)

Rows:  2640037
Three sample reviews:


,star_rating,review_headline,review_body
246582,5,Fast Shipping!,Fast Shipping. Works Perfectly!
2639050,5,it works!,5 stars based on transmission clarity. compar...
697475,5,Five Stars,"Love it, looks great on my wall!"


# data statistics

In [52]:
data['star_rating'] = pd.to_numeric(data['star_rating'], errors='coerce')
print("All reviews: ", data.shape[0])
print(f"1 rating reviews: {data[data['star_rating'] == 1].shape[0]}, {100 * data[data['star_rating'] == 1].shape[0]/data.shape[0]}%")
print(f"2 rating reviews: {data[data['star_rating'] == 2].shape[0]}, {100 * data[data['star_rating'] == 2].shape[0]/data.shape[0]}%")
print(f"3 rating reviews: {data[data['star_rating'] == 3].shape[0]}, {100 * data[data['star_rating'] == 3].shape[0]/data.shape[0]}%")
print(f"4 rating reviews: {data[data['star_rating'] == 4].shape[0]}, {100 * data[data['star_rating'] == 4].shape[0]/data.shape[0]}%")
print(f"5 rating reviews: {data[data['star_rating'] == 5].shape[0]}, {100 * data[data['star_rating'] == 5].shape[0]/data.shape[0]}%")

All reviews:  2640037
1 rating reviews: 306962, 11.627185528081615%
2 rating reviews: 138380, 5.241593204943719%
3 rating reviews: 193674, 7.336033548014668%
4 rating reviews: 418339, 15.845952159003833%
5 rating reviews: 1582682, 59.949235559956165%


 ## We form three classes and select 100000 reviews randomly from positive and negtive class.



In [53]:
pos_reviews = data[data['star_rating'] > 3]
neg_reviews = data[data['star_rating'] <= 2]
neu_reviews = data[data['star_rating'] == 3]
print("All reviews: ", data.shape[0])
print("Positive reviews: ", pos_reviews.shape[0])
print("Negative reviews: ", neg_reviews.shape[0])
print("Neutral reviews: ", neu_reviews.shape[0])
print("All reviews == Positive + Negative + Neutral: ", data.shape[0] == pos_reviews.shape[0] + neg_reviews.shape[0] + neu_reviews.shape[0])

All reviews:  2640037
Positive reviews:  2001021
Negative reviews:  445342
Neutral reviews:  193674
All reviews == Positive + Negative + Neutral:  True


In [54]:
pos_samples = pos_reviews.sample(n=100000, random_state=0)
neg_samples = neg_reviews.sample(n=100000, random_state=0)
pos_samples['star_rating'] = 1
neg_samples['star_rating'] = 0
pos_samples.rename(columns={'star_rating': 'label'}, inplace=True)
neg_samples.rename(columns={'star_rating': 'label'}, inplace=True)
dataset = pd.concat([pos_samples, neg_samples])
dataset['review'] = dataset[['review_headline', 'review_body']].agg(' '.join, axis=1)
dataset.drop(columns=['review_headline', 'review_body'], inplace=True)
print("Rows: ", dataset.shape[0])
print("Three sample reviews:")
dataset.sample(3, random_state=1)

Rows:  200000
Three sample reviews:


,label,review
1448408,1,Works GREAT! So many great uses! Initially wa...
2070049,0,not good ink. Toner ink is not clear and look ...
2471912,1,An almost perfect product! The Bic Wite-Out Co...


# Data Cleaning



In [55]:
print("Average length of reviews before cleaning: ", dataset['review'].str.len().mean())
print("Three sample reviews:")
dataset.sample(3,random_state=1)

Average length of reviews before cleaning:  343.482135
Three sample reviews:


,label,review
1448408,1,Works GREAT! So many great uses! Initially wa...
2070049,0,not good ink. Toner ink is not clear and look ...
2471912,1,An almost perfect product! The Bic Wite-Out Co...


In [56]:
from bs4 import MarkupResemblesLocatorWarning
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
dataset['review'] = dataset['review'].apply(str.lower)
dataset['review'] = dataset['review'].apply(lambda x: re.sub(r'https?://\S+|www\.\S+', '', x))
dataset['review'] = dataset['review'].apply(lambda x: BeautifulSoup(x, "html.parser").text)
dataset['review'] = dataset['review'].apply(lambda x: re.sub(r'[^a-zA-Z]+', ' ', x))
dataset['review'] = dataset['review'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())
dataset['review'] = dataset['review'].apply(lambda x: ' '.join([contractions.fix(word) for word in x.split()]))

print("Average length of reviews after cleaning: ", dataset['review'].str.len().mean())
print("Three sample reviews:")
dataset.sample(3,random_state=1)

Average length of reviews after cleaning:  326.426925
Three sample reviews:


,label,review
1448408,1,works great so many great uses initially was b...
2070049,0,not good ink toner ink is not clear and look l...
2471912,1,an almost perfect product the bic wite out cor...


# Pre-processing

In [57]:
print("Average length of reviews before preprocessing: ", dataset['review'].str.len().mean())
print("Three sample reviews:")
dataset.sample(3,random_state=1)

Average length of reviews before preprocessing:  326.426925
Three sample reviews:


,label,review
1448408,1,works great so many great uses initially was b...
2070049,0,not good ink toner ink is not clear and look l...
2471912,1,an almost perfect product the bic wite out cor...


## remove the stop words 

In [58]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stopwords = set(stopwords.words('english'))
dataset['review'] = dataset['review'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stopwords]))
print("Three sample reviews:")
dataset.sample(3,random_state=1)

Three sample reviews:


,label,review
1448408,1,works great many great uses initially bought b...
2070049,0,good ink toner ink clear look like dry ink pri...
2471912,1,almost perfect product bic wite correction tap...


## perform lemmatization  

In [59]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
dataset['review'] = dataset['review'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(x)]))
print("Average length of reviews after preprocessing: ", dataset['review'].str.len().mean())
print("Three sample reviews:")
dataset.sample(3,random_state=1)

Average length of reviews after preprocessing:  204.287135
Three sample reviews:


,label,review
1448408,1,work great many great us initially bought barc...
2070049,0,good ink toner ink clear look like dry ink pri...
2471912,1,almost perfect product bic wite correction tap...


# TF-IDF Feature Extraction

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
vectors = vectorizer.fit_transform(dataset['review'])
print(vectors.shape)

(200000, 6740105)


# Train-Test Split

In [61]:
from sklearn.model_selection import train_test_split
labels = dataset['label']
x_train, x_test, y_train, y_test = train_test_split(vectors, labels, test_size=0.2, random_state=42, stratify=labels)
print("Training set size: ", x_train.shape, y_test.shape)
print("Test set size: ", x_test.shape, y_test.shape)

Training set size:  (160000, 6740105) (40000,)
Test set size:  (40000, 6740105) (40000,)


# Statistics Printing

In [62]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def summary(y_train, pred_train, y_test, pred_test):
    print("Train accuracy: ", accuracy_score(y_train, pred_train))
    print("Train precision: ", precision_score(y_train, pred_train))
    print("Train recall: ", recall_score(y_train, pred_train))
    print("Train f1 score: ", f1_score(y_train, pred_train))
    print("Test accuracy: ", accuracy_score(y_test, pred_test))
    print("Test precision: ", precision_score(y_test, pred_test))
    print("Test recall: ", recall_score(y_test, pred_test))
    print("Test f1 score: ", f1_score(y_test, pred_test))

# Perceptron

In [63]:
from sklearn.linear_model import Perceptron
perceptron = Perceptron()
perceptron.fit(x_train, y_train)

Perceptron()

In [64]:
pred_train = perceptron.predict(x_train)
pred_test = perceptron.predict(x_test)
summary(y_train, pred_train, y_test, pred_test)

Train accuracy:  0.999125
Train precision:  0.9985765657776446
Train recall:  0.999675
Train f1 score:  0.9991254809854581
Test accuracy:  0.931675
Test precision:  0.9258199753390876
Test recall:  0.93855
Test f1 score:  0.9321415270018621


# SVM

In [65]:
from sklearn.svm import LinearSVC
svm = LinearSVC(dual='auto', random_state=0)
svm.fit(x_train, y_train)

LinearSVC(dual='auto', random_state=0)

In [66]:
pred_train = svm.predict(x_train)
pred_test = svm.predict(x_test)
summary(y_train, pred_train, y_test, pred_test)

Train accuracy:  0.99944375
Train precision:  0.999225338914225
Train recall:  0.9996625
Train f1 score:  0.9994438716530759
Test accuracy:  0.937675
Test precision:  0.9408713170486024
Test recall:  0.93405
Test f1 score:  0.9374482499059089


# Logistic Regression

In [67]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(solver='saga', random_state=0)
logit.fit(x_train, y_train)

LogisticRegression(random_state=0, solver='saga')

In [68]:
pred_train = logit.predict(x_train)
pred_test = logit.predict(x_test)
summary(y_train, pred_train, y_test, pred_test)

Train accuracy:  0.9618125
Train precision:  0.9673742536180548
Train recall:  0.9558625
Train f1 score:  0.9615839243498818
Test accuracy:  0.92725
Test precision:  0.9341089209510263
Test recall:  0.91935
Test f1 score:  0.9266706985182944


# Naive Bayes

In [69]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(x_train, y_train)

MultinomialNB()

In [70]:
pred_train = naive_bayes.predict(x_train)
pred_test = naive_bayes.predict(x_test)
summary(y_train, pred_train, y_test, pred_test)

Train accuracy:  0.97436875
Train precision:  0.9905254313966264
Train recall:  0.9579
Train f1 score:  0.9739395672481175
Test accuracy:  0.910625
Test precision:  0.9422693736873283
Test recall:  0.87485
Test f1 score:  0.9073089787134745
